In [6]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
from keras.models import load_model
import tensorflow as tf
import numpy as np

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
char_core = ['acc','agr','beta','bm','cash','cashpr','cfp','chatoia','chcsho','chfeps','chinv','chmom',
             'chpmia','chtx','currat','depr','dy','ear','ep','gma','grcapx','grltnoa','ill','indmom','invest',
             'lev','lgr','maxret','mom12m','mom1m','mom36m','mve','nincr','orgcap','pchgm_pchsale','pchsale_pchinvt',
             'pchsale_pchrect','pchsale_pchxsga','retvol','roaq','roavol','roeq','salecash','saleinv','sgr','sp',
             'std_dolvol','std_turn','turn']

core = ['date','permno','ticker','comnam','exchcd','exchname','siccd',
        'indname','size_class','mve_m','ret_adj_ex',]

#core = ['fyear','year','jyear','permno','ticker','comnam','exchcd','exchname','siccd',
#        'indname','size_class','mve_m','rf','ret','ret_adj','ret_ex','ret_adj_ex',]

print(len(char_core), len(core))

data = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Dataset/research_result/norm_df49.pkl')
data['date'] = pd.to_datetime(data.index).to_period('M')
data.sort_values(by='date', ascending=True, inplace=True)

X = data[char_core+['date']]
X_info = data[core]
Y = data[['predicted_return','date']]
print(X.shape, Y.shape)

49 11
(210205, 50) (210205, 2)


In [15]:
def my_metric_fn(y_true, y_pred):
    num = tf.reduce_mean(tf.square(y_true - y_pred))
    den = tf.reduce_mean(tf.square(y_true))
    return 1 - num / den

model_file = '/content/drive/MyDrive/Colab Notebooks/Dataset/research_result/model2.h5'

model = load_model(model_file, custom_objects = {"my_metric_fn": my_metric_fn})

In [ ]:
import keras
y_hat_all = []
mse_ = {}
r2_ = {}

for i in range(2000, 2023):
    print(f"Year {i}")
    x_train = X[ (X['date'] < pd.Period((str(i)+"-1"),freq='M')) ]
    y_train = Y[ (Y['date'] < pd.Period((str(i)+"-1"),freq='M')) ]

    x_test = X[(X['date'] >= pd.Period((str(i)+"-1"),freq='M')) & (X['date'] <= pd.Period((str(i)+"-12"),freq='M'))]
    y_test = Y[(X['date'] >= pd.Period((str(i)+"-1"),freq='M')) & (X['date'] <= pd.Period((str(i)+"-12"),freq='M'))]

    x_train = x_train.drop("date", axis=1)
    y_train = y_train.drop("date", axis=1)
    x_test = x_test.drop("date", axis=1)
    y_test = y_test.drop("date", axis=1)

    model_1yr = load_model(model_file, custom_objects = {"my_metric_fn": my_metric_fn})
    model_1yr.fit(x_train, y_train, epochs=30,
                  callbacks=[keras.callbacks.EarlyStopping(monitor='loss', patience=2)],  batch_size = 256,)
    mse, r2, mae = model_1yr.evaluate(x_test, y_test)
    print(mse, r2, mae)
    mse_[i] =mse
    r2_[i] = r2
    y_NN2 = model_1yr.predict(x_test)
    for a in y_NN2.reshape(-1):
        y_hat_all.append(a)
    print(y_NN2.reshape(-1).mean(), y_NN2.reshape(-1).std())

In [ ]:
# Linear Model of Senario 2
y_hat_linear = []
mse_linear = []
r2_linear = []

for i in range(2000, 2023):
    print(f"Year {i}")
    x_train = X[ (X['date'] < pd.Period((str(i)+"-1"),freq='M')) ]
    y_train = Y[ (Y['date'] < pd.Period((str(i)+"-1"),freq='M')) ]

    x_test = X[(X['date'] >= pd.Period((str(i)+"-1"),freq='M')) & (X['date'] <= pd.Period((str(i)+"-12"),freq='M'))]
    y_test = Y[(X['date'] >= pd.Period((str(i)+"-1"),freq='M')) & (X['date'] <= pd.Period((str(i)+"-12"),freq='M'))]

    x_train = x_train.drop("date", axis=1)
    y_train = y_train.drop("date", axis=1)
    x_test = x_test.drop("date", axis=1)
    y_test = y_test.drop("date", axis=1)

    regr = linear_model.LinearRegression()
    regr.fit(x_train, y_train)
    y_pred_linear = regr.predict(x_test)
    mse = mean_squared_error(y_test, y_pred_linear)
    for a in y_pred_linear.reshape(-1):
        y_hat_linear.append(a)

In [18]:
X_info = data[core]
x_test_info = X_info[(X_info['date'] >= pd.Period((str(2000)+"-1"),freq='M'))]

x_test = X[(X['date'] >= pd.Period((str(2000)+"-1"),freq='M'))]
y_test = Y[(Y['date'] >= pd.Period((str(2000)+"-1"),freq='M'))]

senario_2_result = x_test_info.copy()
senario_2_result.index = x_test.index

senario_2_result['y_real'] = y_test['predicted_return']
senario_2_result['NN_pred_pretrain'] = y_hat_all
senario_2_result['lin_pred_pretrain'] = y_hat_linear

senario_2_result.to_pickle("senario_2_result1.pkl")

In [19]:
## MSE
mse_Linear = mean_squared_error(senario_2_result['y_real'], senario_2_result['lin_pred_pretrain'])
mse_NN = mean_squared_error(senario_2_result['y_real'], senario_2_result['NN_pred_pretrain'])
print(f"【Senario 2】MSE of Linear model: {mse_Linear}")
print(f"【Senario 2】MSE of Neural model: {mse_NN}")

【Senario 2】MSE of Linear model: 0.01566540810352375
【Senario 2】MSE of Neural model: 0.015703034167274834


In [20]:
# R^2 of Linear Regresion
a = np.sum(np.square(senario_2_result['y_real'] -  senario_2_result['lin_pred_pretrain']))
b = np.sum(np.square(senario_2_result['y_real']))
print(f"【Senario 2】 R^2 of Linear Regresion :",1-a/b)

# R^2 of Neural Network
a = np.sum(np.square(senario_2_result['y_real'] -  senario_2_result['NN_pred_pretrain']))
b = np.sum(np.square(senario_2_result['y_real']))
print(f"【Senario 2】 R^2 of Neural Network :",1-a/b)

【Senario 2】 R^2 of Linear Regresion : 0.0057037489309388345
【Senario 2】 R^2 of Neural Network : 0.003315591923923278


In [21]:
# Metrics: XS-R^2 of Neural Network Model  (weighted)
stock_list = senario_2_result['permno'].unique()
month_list = senario_2_result.index.unique()
print(len(stock_list), len(month_list))

r2_stock = {}
num_ = []
deno_ = []
for i in stock_list:
  df = senario_2_result[senario_2_result['permno'] == i]
  num_month = df.shape[0]
  num = np.square((df['y_real'] -  df['NN_pred_pretrain']).sum() / num_month) * num_month
  deno = np.square((df['NN_pred_pretrain']).sum() / num_month)  * num_month
  num_.append(num)
  deno_.append(deno)
print(f"XS-R^2 of Neural Network Model  (weighted):",1 - np.mean(num_) / np.mean(deno_))

# Metrics: XS-R^2 of Linear Regresion (weighted)
r2_stock = {}
num_ = []
deno_ = []
for i in stock_list:
  df = senario_2_result[senario_2_result['permno'] == i]
  num_month = df.shape[0]
  num = np.square((df['y_real'] -  df['lin_pred_pretrain']).sum() / num_month) * num_month
  deno = np.square((df['lin_pred_pretrain']).sum() / num_month)  * num_month
  num_.append(num)
  deno_.append(deno)
print(f"XS-R^2 of Linear Regresion (weighted):", 1 - np.mean(num_) / np.mean(deno_))

607 269
XS-R^2 of Neural Network Model  (weighted): -0.7404501948643314
XS-R^2 of Linear Regresion (weighted): -0.9542138628254369


In [24]:
# Metrics: XS-R^2 of Neural Network Model  (no weighted)
stock_list = senario_2_result['permno'].unique()
month_list = senario_2_result.index.unique()
print(len(stock_list), len(month_list))

r2_stock = {}
num_ = []
deno_ = []
for i in stock_list:
  df = senario_2_result[senario_2_result['permno'] == i]
  num_month = df.shape[0]
  num = np.square((df['y_real'] -  df['NN_pred_pretrain']).sum() / num_month)
  deno = np.square((df['NN_pred_pretrain']).sum() / num_month)
  num_.append(num)
  deno_.append(deno)
print(f"【Senario 2】 XS-R^2 of Neural Network Model  (no weighted):",1 - np.mean(num_) / np.mean(deno_))

# Metrics: XS-R^2 of Linear Regresion (no weighted)
r2_stock = {}
num_ = []
deno_ = []
for i in stock_list:
  df = senario_2_result[senario_2_result['permno'] == i]
  num_month = df.shape[0]
  num = np.square((df['y_real'] -  df['lin_pred_pretrain']).sum() / num_month)
  deno = np.square((df['lin_pred_pretrain']).sum() / num_month)
  num_.append(num)
  deno_.append(deno)
print(f"【Senario 2】 XS-R^2 of Linear Regresion (no weighted):", 1 - np.mean(num_) / np.mean(deno_))

607 269
【Senario 2】 XS-R^2 of Neural Network Model  (no weighted): -19.60326057898728
【Senario 2】 XS-R^2 of Linear Regresion (no weighted): -23.84399135612912


In [22]:
# Metrics: EV of Neural Network
r2_stock = {}
num_ = []
deno_ = []
for i in month_list:
  df = senario_2_result[senario_2_result.index == i]
  num_stock = df.shape[0]
  num = (np.square(df['y_real'] -  df['NN_pred_pretrain'])).sum() / num_stock
  deno = (np.square(df['y_real'])).sum() /num_stock
  num_.append(num)
  deno_.append(deno)
  break
print(f"【Senario 2】 EV of Neural Network:",1 - np.mean(num_) / np.mean(deno_))

# Metrics: EV of Linear Model
r2_stock = {}
num_ = []
deno_ = []
for i in month_list:
  df = senario_2_result[senario_2_result.index == i]
  num_stock = df.shape[0]
  num = (np.square(df['y_real'] -  df['lin_pred_pretrain'])).sum() / num_stock
  deno = (np.square(df['y_real'])).sum() /num_stock
  num_.append(num)
  deno_.append(deno)
  break
print(f"【Senario 2】 EV of Linear Model:",1 - np.mean(num_) / np.mean(deno_))

【Senario 2】 EV of Neural Network: -0.00010379938953031242
【Senario 2】 EV of Linear Model: 0.004059510189341364


In [ ]:
# Neural Network

import keras
y_hat_all = []
mse_ = {}
r2_ = {}

for i in range(2000, 2023):
    print(f"Year {i}")
    x_train = X[ (X['date'] < pd.Period((str(i)+"-1"),freq='M')) ]
    y_train = Y[ (Y['date'] < pd.Period((str(i)+"-1"),freq='M')) ]

    x_test = X[(X['date'] >= pd.Period((str(i)+"-1"),freq='M')) & (X['date'] <= pd.Period((str(i)+"-12"),freq='M'))]
    y_test = Y[(X['date'] >= pd.Period((str(i)+"-1"),freq='M')) & (X['date'] <= pd.Period((str(i)+"-12"),freq='M'))]

    x_train = x_train.drop("date", axis=1)
    y_train = y_train.drop("date", axis=1)
    x_test = x_test.drop("date", axis=1)
    y_test = y_test.drop("date", axis=1)

    model_1yr = load_model(model_file, custom_objects = {"my_metric_fn": my_metric_fn})
    model_1yr.fit(x_train, y_train, epochs=30,
                  callbacks=[keras.callbacks.EarlyStopping(monitor='loss', patience=2)],  batch_size = 256,)
    mse, r2, mae = model_1yr.evaluate(x_test, y_test)
    print(mse, r2, mae)
    mse_[i] =mse
    r2_[i] = r2
    y_NN2 = model_1yr.predict(x_test)
    for a in y_NN2.reshape(-1):
        y_hat_all.append(a)
    print(y_NN2.reshape(-1).mean(), y_NN2.reshape(-1).std())

In [ ]:
# Linear Model of Senario 2
y_hat_linear = []
mse_linear = []
r2_linear = []

for i in range(2000, 2023):
    print(f"Year {i}")
    x_train = X[ (X['date'] < pd.Period((str(i)+"-1"),freq='M')) ]
    y_train = Y[ (Y['date'] < pd.Period((str(i)+"-1"),freq='M')) ]

    x_test = X[(X['date'] >= pd.Period((str(i)+"-1"),freq='M')) & (X['date'] <= pd.Period((str(i)+"-12"),freq='M'))]
    y_test = Y[(X['date'] >= pd.Period((str(i)+"-1"),freq='M')) & (X['date'] <= pd.Period((str(i)+"-12"),freq='M'))]

    x_train = x_train.drop("date", axis=1)
    y_train = y_train.drop("date", axis=1)
    x_test = x_test.drop("date", axis=1)
    y_test = y_test.drop("date", axis=1)

    regr = linear_model.LinearRegression()
    regr.fit(x_train, y_train)
    y_pred_linear = regr.predict(x_test)
    mse = mean_squared_error(y_test, y_pred_linear)
    for a in y_pred_linear.reshape(-1):
        y_hat_linear.append(a)

In [8]:
X_info = data[core]
x_test_info = X_info[(X_info['date'] >= pd.Period((str(2000)+"-1"),freq='M'))]

x_test = X[(X['date'] >= pd.Period((str(2000)+"-1"),freq='M'))]
y_test = Y[(Y['date'] >= pd.Period((str(2000)+"-1"),freq='M'))]

senario_2_result = x_test_info.copy()
senario_2_result.index = x_test.index

senario_2_result['y_real'] = y_test['predicted_return']
senario_2_result['NN_pred_pretrain'] = y_hat_all
senario_2_result['lin_pred_pretrain'] = y_hat_linear

senario_2_result.to_pickle("senario_2_result.pkl")

In [9]:
## MSE
mse_Linear = mean_squared_error(senario_2_result['y_real'], senario_2_result['lin_pred_pretrain'])
mse_NN = mean_squared_error(senario_2_result['y_real'], senario_2_result['NN_pred_pretrain'])
print(f"【Senario 2】MSE of Linear model: {mse_Linear}")
print(f"【Senario 2】MSE of Neural model: {mse_NN}")
print()

# R^2 of Linear Regresion
a = np.sum(np.square(senario_2_result['y_real'] -  senario_2_result['lin_pred_pretrain']))
b = np.sum(np.square(senario_2_result['y_real']))
print(f"【Senario 2】 R^2 of Linear Regresion :",1-a/b)

# R^2 of Neural Network
a = np.sum(np.square(senario_2_result['y_real'] -  senario_2_result['NN_pred_pretrain']))
b = np.sum(np.square(senario_2_result['y_real']))
print(f"【Senario 2】 R^2 of Neural Network :",1-a/b)

【Senario 2】MSE of Linear model: 0.01566540810352375
【Senario 2】MSE of Neural model: 0.0156932345213041


In [10]:
# R^2 of Linear Regresion
a = np.sum(np.square(senario_2_result['y_real'] -  senario_2_result['lin_pred_pretrain']))
b = np.sum(np.square(senario_2_result['y_real']))
print(f"【Senario 2】 R^2 of Linear Regresion :",1-a/b)

# R^2 of Neural Network
a = np.sum(np.square(senario_2_result['y_real'] -  senario_2_result['NN_pred_pretrain']))
b = np.sum(np.square(senario_2_result['y_real']))
print(f"【Senario 2】 R^2 of Neural Network :",1-a/b)

【Senario 2】 R^2 of Linear Regresion : 0.0057037489309388345
【Senario 2】 R^2 of Neural Network : 0.003937583460059013


In [12]:
# Metrics: XS-R^2 of Neural Network Model  (weighted)
stock_list = senario_2_result['permno'].unique()
month_list = senario_2_result.index.unique()
print(len(stock_list), len(month_list))

r2_stock = {}
num_ = []
deno_ = []
for i in stock_list:
  df = senario_2_result[senario_2_result['permno'] == i]
  num_month = df.shape[0]
  num = np.square((df['y_real'] -  df['NN_pred_pretrain']).sum() / num_month) * num_month
  deno = np.square((df['NN_pred_pretrain']).sum() / num_month)  * num_month
  num_.append(num)
  deno_.append(deno)
print(f"XS-R^2 of Neural Network Model  (weighted):",1 - np.mean(num_) / np.mean(deno_))

# Metrics: XS-R^2 of Linear Regresion (weighted)
r2_stock = {}
num_ = []
deno_ = []
for i in stock_list:
  df = senario_2_result[senario_2_result['permno'] == i]
  num_month = df.shape[0]
  num = np.square((df['y_real'] -  df['lin_pred_pretrain']).sum() / num_month) * num_month
  deno = np.square((df['lin_pred_pretrain']).sum() / num_month)  * num_month
  num_.append(num)
  deno_.append(deno)
print(f"XS-R^2 of Linear Regresion (weighted):", 1 - np.mean(num_) / np.mean(deno_))

XS-R^2 of Neural Network Model  (weighted): -0.1614102688995509
XS-R^2 of Linear Regresion (weighted): -0.9542138628254369


In [13]:
# Metrics: EV of Neural Network
r2_stock = {}
num_ = []
deno_ = []
for i in month_list:
  df = senario_2_result[senario_2_result.index == i]
  num_stock = df.shape[0]
  num = (np.square(df['y_real'] -  df['NN_pred_pretrain'])).sum() / num_stock
  deno = (np.square(df['y_real'])).sum() /num_stock
  num_.append(num)
  deno_.append(deno)
  break
print(f"【Senario 2】 EV of Neural Network:",1 - np.mean(num_) / np.mean(deno_))

# Metrics: EV of Linear Model
r2_stock = {}
num_ = []
deno_ = []
for i in month_list:
  df = senario_2_result[senario_2_result.index == i]
  num_stock = df.shape[0]
  num = (np.square(df['y_real'] -  df['lin_pred_pretrain'])).sum() / num_stock
  deno = (np.square(df['y_real'])).sum() /num_stock
  num_.append(num)
  deno_.append(deno)
  break
print(f"【Senario 2】 EV of Linear Model:",1 - np.mean(num_) / np.mean(deno_))

【Senario 2】 EV of Neural Network: -0.0011904280992580674
【Senario 2】 EV of Linear Model: 0.004059510189341364
